***This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.***

## 1
Read the case, department, and source data into their own spark dataframes.

## 2
Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

## 3
Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [1]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType


# creating a spark
spark = SparkSession.builder.getOrCreate()

# 1
Read the case, department, and source data into their own spark dataframes.

In [2]:
# read the case.csv
case_df = spark.read.csv("case.csv", header=True, inferSchema=True)

In [3]:
# look at the case_df
case_df.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [4]:
# read the source.csv
source_df = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [5]:
# look at the source_df
source_df.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [6]:
# reading the dept.csv
dept_df = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)


In [7]:
# look at the dept_df
dept_df.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



# 2
Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
?

In [8]:
schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)
# Read csv, but now we specify the schema:
source_df = spark.read.csv("source.csv", header=True, schema=schema)

In [9]:
# write data to a destination using .write property to jason 

source_df.write.json('source_json', mode= 'overwrite')
dept_df.write.json('dept_json', mode = 'overwrite')
case_df.write.json('case_json', mode = 'overwrite')

## 2b
Inspect your folder structure. What do you notice?

- There is a folder with:
    - A text file that success 
    - A jason file with all info

## 3
Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [10]:
# check source data tyeps
source_df.printSchema()
# sourece data types = ok

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [11]:
case_df.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [12]:
case_df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



## Takeaway 
**case_df**
- change date to date time 
- cahnge case_late, case_close
- change num days late to float
- change council_district to string

In [13]:
# rename this column to match with the other date-type columns
case_df = case_df.withColumnRenamed("SLA_due_date", "case_due_date")

In [14]:
# Two columns, case_closed and case_late store yes/no values. let's turn them into booleans:
case_df = case_df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"'))


In [15]:
# demonstrating we only have yes/no in each field
case_df.groupBy("case_closed", "case_late").count().show()


+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|       true|    false|735616|
|       true|     true| 87978|
|      false|    false| 11585|
|      false|     true|  6525|
+-----------+---------+------+



In [16]:
# change council_district dtype to string 
case_df = case_df.withColumn('council_district', col('council_district').cast('string'))


In [19]:
print("--- Before handling dates")
case_df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
case_df = (
    case_df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
case_df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)


--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:

In [21]:
# normalizing the request address field. Using the trim and lower functions lets us strip any leading or trailing whitespace and convert everything to lowercase.

print("--- Before")
case_df.select("request_address").show(5)

case_df = case_df.withColumn("request_address", trim(lower(case_df.request_address)))

print("--- After")
case_df.select("request_address").show(5)


--- Before
+--------------------+
|     request_address|
+--------------------+
|2315  EL PASO ST,...|
|2215  GOLIAD RD, ...|
|102  PALFREY ST W...|
|114  LA GARDE ST,...|
|734  CLEARVIEW DR...|
+--------------------+
only showing top 5 rows

--- After
+--------------------+
|     request_address|
+--------------------+
|2315  el paso st,...|
|2215  goliad rd, ...|
|102  palfrey st w...|
|114  la garde st,...|
|734  clearview dr...|
+--------------------+
only showing top 5 rows



In [23]:
# convert the number of days a case is late to a number of weeks.
case_df = case_df.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

case_df.select("num_days_late", "num_weeks_late").show(5)


+-------------------+--------------------+
|      num_days_late|      num_weeks_late|
+-------------------+--------------------+
| -998.5087616000001|        -142.6441088|
|-2.0126041669999997|-0.28751488099999994|
|       -3.022337963|-0.43176256614285713|
|       -15.01148148| -2.1444973542857144|
|0.37216435200000003|         0.053166336|
+-------------------+--------------------+
only showing top 5 rows



In [24]:
# ormat the council district column a little differently. We'll add leading 0s to it
case_df = case_df.withColumn("council_district", col("council_district").cast("int"))

# '%03d' means at least 3 digits, pad with 0s
#
# In order to use the format_string function the way we are, we'll need to
# convert council_district back to an integer temporarily, but the final output
# will be a string.
case_df = case_df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

case_df.select("council_district").show(5)


+----------------+
|council_district|
+----------------+
|             005|
|             003|
|             003|
|             003|
|             007|
+----------------+
only showing top 5 rows



In [25]:
# create some new features based on our existing data
# extract the zipcode from the address
case_df = case_df.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))

case_df.select("zipcode").show(5)


+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
|  78223|
|  78228|
+-------+
only showing top 5 rows



In [27]:
case_df = (
    case_df.withColumn(
        # case_age: How old the case is; the difference in days between when the case was opened and the current day
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        #days_to_closed: The number of days between when the case was opened and when it was closed
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        #case_lifetime: Number of days between when the case was opened and when it was closed, if the case is still open, the number of days since the case was opened
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

case_df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

case_df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)


+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 00:42:00|    1505|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-01 00:46:00|    1505|             0|            0|
|       true|2018-01-01 00:48:00|2018-01-01 00:48:00|    1505|             0|            0|
|       true|2018-01-01 01:29:00|2018-01-01 01:29:00|    1505|             0|            0|
|       true|2018-01-01 01:34:00|2018-01-01 01:34:00|    1505|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|ca

In [30]:
case_df.show(2, vertical=True)


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1505                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127333

In [31]:
# we can join it to our case dataframe using the dept_division column.
case_df = (
    case_df
    # left join on dept_division
    .join(dept_df, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept_df.dept_division)
    .drop(dept_df.dept_name)
    .drop(case_df.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

case_df.show(2, vertical=True)


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1505                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1-------------------------

## Train Test Split

In [ ]:
# train, test = df.randomSplit([0.8, 0.2])
# #
# train, validate, test = df.randomSplit([0.6, 0.2, 0.2])


----------------------------------------

### 1
How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

### 2
How many Stray Animal cases are there?

### 3
How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

### 4
Convert the council_district column to a string column.

### 5
Extract the year from the `case_closed_date` column.

### 6
Convert num_days_late from days to hours in new columns `num_hours_late`.

### 7
Join the case data with the source and department data.

### 8
Are there any cases that do not have a request source?

### 9
What are the top 10 service request types in terms of number of requests?

### 10
What are the top 10 service request types in terms of average days late?

### 11
Does number of days late depend on department?

### 12
How do number of days late depend on department and request type?

In [32]:
df = case_df

## 1
How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [33]:
(
    df.select(datediff(current_timestamp(), 'case_due_date')
    .alias('days_past_due'))
    .where(df.case_closed == False)
    .sort(col('days_past_due').desc())
    .show(5)
)

+-------------+
|days_past_due|
+-------------+
|         1870|
|         1870|
|         1869|
|         1868|
|         1867|
+-------------+
only showing top 5 rows



### 2
How many Stray Animal cases are there?

In [34]:
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+



### 3
How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [36]:
case_df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1505                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [40]:
(
    df.filter(df.department == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

0

### 4
Convert the council_district column to a string column.

In [41]:
case_df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



### 5
Extract the year from the `case_closed_date` column.

In [44]:
df.select(year('case_closed_date')).show(5)

+----------------------+
|year(case_closed_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 5 rows



### 6
Convert num_days_late from days to hours in new columns `num_hours_late`.

In [47]:
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show(5)
)


+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



### 7
Join the case data with the source and department data.

In [56]:
# join source table
df = df.join(source_df, on = df.source_id== source_df.source_id).drop(source_df.source_id)


In [58]:
df.show(2, truncate = False, vertical = True)


-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 00:42:00                  
 case_due_date        | 2018-01-01 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 num_weeks_late       | -142.6441088                         
 zipcode              | 78207                                
 case_ag

### 8
Are there any cases that do not have a request source?

In [63]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [64]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



### 9
What are the top 10 service request types in terms of number of requests?


In [66]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### 10
What are the top 10 service request types in terms of average days late?


In [67]:
# group by service type request, 
df.filter(col('case_late') == 'true')\
.groupBy('service_request_type').agg(mean('num_days_late').alias('avg_days_late')).sort(desc('avg_days_late')).show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg_days_late     |
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942615|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



### 11
Does number of days late depend on department?

In [68]:
df.filter(col('case_late') == 'true')\
.groupby('department').agg(round(mean('num_days_late'),2).alias('avg_days_late'))\
.sort(desc('avg_days_late')).show(truncate = False)

+------------------------+-------------+
|department              |avg_days_late|
+------------------------+-------------+
|Customer Service        |88.18        |
|DSD/Code Enforcement    |49.51        |
|Animal Care Services    |23.45        |
|Parks & Recreation      |22.43        |
|Trans & Cap Improvements|10.66        |
|Solid Waste             |7.15         |
|Metro Health            |6.49         |
+------------------------+-------------+



### 12
How do number of days late depend on department and request type?

In [72]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)
 |-- source_username: string (nullable = true)



In [73]:
(
    df.filter("case_closed")
#     .filter("case_late")
    .groupby("department", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .where(col('days_late') > 0)
    .sort(desc("days_late"))
    .show(40, truncate=False)
)

+------------------------+----------------------------------------+---------+-------+
|department              |service_request_type                    |days_late|n_cases|
+------------------------+----------------------------------------+---------+-------+
|DSD/Code Enforcement    |Zoning: Junk Yards                      |209.4    |174    |
|DSD/Code Enforcement    |Labeling for Used Mattress              |162.4    |7      |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses       |154.0    |7      |
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr  |151.6    |12     |
|DSD/Code Enforcement    |Storage of Used Mattress                |142.1    |8      |
|DSD/Code Enforcement    |Donation Container Enforcement          |141.3    |114    |
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |139.0    |141    |
|DSD/Code Enforcement    |License Requied Used Mattress Sales     |128.8    |7      |
|DSD/Code Enforcement    |Vendors                     